In [1]:
import tensorflow as tf
import os

In [3]:
#Avoid OOm errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Remove Dodgy Images

In [5]:
import cv2
import imghdr

In [7]:
data_dir = 'data'
image_exts = ['jpeg','jpg','bmp','png']

In [8]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path = os.path.join(data_dir,image_class,image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("issue with image {}".format((image_path)))

Image not in ext list data\happy_people\220px-Happy_People_A_Year_in_the_Taiga_poster.jpg
Image not in ext list data\happy_people\2560px-Happy_People.jpg
Image not in ext list data\happy_people\427394C200000578-0-image-a-24_1500384481554.jpg
Image not in ext list data\happy_people\6-Habits-Of-Happy-People.jpg
Image not in ext list data\happy_people\depositphotos_391577690-stock-photo-cheerful-woman-outstretched-hands-sitting.jpg
Image not in ext list data\happy_people\guilherme-stecanella-375176-unsplash.jpg
Image not in ext list data\happy_people\how-to-watch-shiny-happy-people-64a2f8ea7dc3014582658-900.webp
Image not in ext list data\happy_people\smiling-young-african-american-woman-260nw-2290057229.jpg
Image not in ext list data\happy_people\traitshappypeople.jpg
Image not in ext list data\sad_people\1cd95b44e0d785987a6596d4baf940fc19-13-sad-white-man.2x.h473.w710.jpg
Image not in ext list data\sad_people\CC_HE_1221887081_SituationalDepression.jpg
Image not in ext list data\sad_peop

Load Dataset

In [11]:
import numpy as np
import matplotlib.pyplot as plt

In [12]:
data = tf.keras.utils.image_dataset_from_directory('data')

Found 157 files belonging to 2 classes.


In [13]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()